In [2]:
from urllib import parse
import pandas as pd
from random import randint

from IPython.display import display
import numpy as np
import re

import glob
import os
import shutil
import math


/home/yehia.elshater/ve3.6.3/Python-3.6.3/Lib/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/yehia.elshater/ve3.6.3/Python-3.6.3/Lib/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
def replace_special_chars (s,rep='_'):
    return re.sub('[^a-zA-Z0-9\n\.]', rep, s)

In [4]:
def get_excel_section_as_df(excel_df,start_section_name,end_section_name='',offset=0,footer_offset=0):
    nrows = excel_df.book.sheet_by_index(0).nrows
    excel_parsed_df = excel_df.parse(0).dropna(axis=1, how='all')
    start_row = excel_parsed_df[excel_parsed_df["Unnamed: 0"]== start_section_name].index[0]+3 + offset
    if end_section_name == '':
        return excel_df.parse(0, skipfooter= 0,skiprows=start_row).dropna(axis=1, how='all') 
    end_row = excel_parsed_df[excel_parsed_df["Unnamed: 0"]== end_section_name].index[0]+1+footer_offset
    return excel_df.parse(0, skipfooter= nrows-(end_row),skiprows=start_row).dropna(axis=1, how='all')


In [5]:
def get_value_if_exists(df,start_column_index,end_column_index,reshape = False):
    if len(df) == 1:
        if reshape == True:
            return df.iloc[:,start_column_index:end_column_index+1].values[0].reshape(start_column_index,end_column_index)
        else:
            return df.iloc[:,start_column_index:end_column_index+1].values[0]
    if reshape == True:
        #return np.zeros(end_column_index).reshape(start_column_index,end_column_index)
        return np.full((start_column_index,end_column_index),-1)
    return -1

In [6]:
np.zeros(3).reshape(1,3)
#np.full((1,3),-1)

array([[0., 0., 0.]])

In [7]:
def check_valid_profile(profile_name_path):
    ccn = profile_name_path.split("/")[-1].split("_")[0]
    profile_xl = pd.ExcelFile(profile_name_path)
    nrows = profile_xl.book.sheet_by_index(0).nrows


    #profile_general_info_df = profile_xl.parse(0, skipfooter= nrows-(43),skiprows=19).dropna(axis=1, how='all')
    profile_general_info_df = get_excel_section_as_df(profile_xl,"CMS Certification Number:").iloc[:,0:2]
    profile_general_info_df.columns = ["measure","value"]
    profile_general_info_cleaned_df = pd.DataFrame(columns=["Type_Of_Facility","Is_Low_Volume_Hospital","Total_Emp","Total_Discharges","Urban_Rural"])


    
    total_employees = profile_general_info_df[profile_general_info_df.measure == "Total Employees:"].value.values[0].replace(',','')
    total_discharges = profile_general_info_df[profile_general_info_df.measure == "Total Discharges:"].value.values[0]

    try:
        if int(total_employees) == 0 and int(total_discharges)==0:
            return False
        return True
    except:
        return False


In [8]:
def get_profile_df(profile_name_path):
    #profile_name_path = "../web_data/ahd/100001_profile_2018-08-15.xls"
    ccn = profile_name_path.split("/")[-1].split("_")[0]
    profile_xl = pd.ExcelFile(profile_name_path)
    nrows = profile_xl.book.sheet_by_index(0).nrows


    #profile_general_info_df = profile_xl.parse(0, skipfooter= nrows-(43),skiprows=19).dropna(axis=1, how='all')
    profile_general_info_df = get_excel_section_as_df(profile_xl,"CMS Certification Number:").iloc[:,0:2]
    profile_general_info_df.columns = ["measure","value"]
    profile_general_info_cleaned_df = pd.DataFrame(columns=["Type_Of_Facility","Is_Low_Volume_Hospital","Total_Emp","Total_Discharges","Urban_Rural"])


    type_of_facility = profile_general_info_df[profile_general_info_df.measure == "Type of Facility:"].value.values[0]
    is_low_volume = profile_general_info_df[profile_general_info_df.measure == "Low Volume Hospital:"].value.values[0]
    total_employees = profile_general_info_df[profile_general_info_df.measure == "Total Employees:"].value.values[0].replace(',','')
    total_discharges = profile_general_info_df[profile_general_info_df.measure == "Total Discharges:"].value.values[0]
    urban_rural = profile_general_info_df[profile_general_info_df.measure == "Urban / Rural Designation:"].value.values[0] 

    profile_general_info_cleaned_df = profile_general_info_cleaned_df.append({'Type_Of_Facility':type_of_facility,
                                           'Is_Low_Volume_Hospital': is_low_volume,
                                           'Total_Emp':total_employees,
                                           'Total_Discharges': total_discharges,
                                           'Urban_Rural':urban_rural },ignore_index=True)
    profile_general_info_cleaned_df["Provider_ID"] = ccn

    #beds_rev_inpatient_days_df = profile_xl.parse(0, skipfooter= nrows-(101),skiprows=92).dropna(axis=1, how='all')
    beds_rev_inpatient_days_df = get_excel_section_as_df(profile_xl,"Utilization Statistics by Payor","Total Hospital",offset=2,footer_offset=2)
    beds_rev_inpatient_days_df.columns = ["Type_Of_Service","beds","revenue","MedicareDays","MedicaidDays","OtherDays","Total"]
    beds_rev_inpatient_days_df = beds_rev_inpatient_days_df[beds_rev_inpatient_days_df.Type_Of_Service == "Total Hospital"]
    beds_rev_inpatient_days_df = beds_rev_inpatient_days_df[["beds","revenue","Total"]]
    beds_rev_inpatient_days_df.rename(columns={"Total":"Total_Inpatient_Days"},inplace=True)
    beds_rev_inpatient_days_df["Provider_ID"] = ccn


    #gross_rev_df = profile_xl.parse(0, skipfooter= nrows-(113),skiprows=111).dropna(axis=1, how='all')
    gross_rev_df = get_excel_section_as_df(profile_xl,"Total Hospital Patient Revenue","HMO Utilization",offset=-2)
    beds_rev_inpatient_days_df["Gross_Patient_Revenue"] = gross_rev_df["Total"][0]

    #estimated_patient_volumes_df = profile_xl.parse(0, skipfooter= nrows-(130),skiprows=123).dropna(axis=1, how='all')
    '''
    estimated_patient_volumes_df = get_excel_section_as_df(profile_xl,"Estimated Patient Volumes","Clinical Services")
    estimated_patient_volumes_df.columns = ["section","volume"]
    estimated_patient_volumes_df = pd.pivot_table(estimated_patient_volumes_df,columns=["section"],values=["volume"],aggfunc=lambda x: 50 if 'Less' in x else x).reset_index()
    estimated_patient_volumes_df.drop(["index"],axis=1)
    estimated_patient_volumes_df.columns = ["s","birth_volume","ER_admitted_volume","ER_not_admitted_volume","ip_surgeries_volume","op_surgeries_volume","op_visits_volume"]
    estimated_patient_volumes_df = estimated_patient_volumes_df.drop(["s"],axis=1)
    estimated_patient_volumes_df["Provider_ID"] = ccn
    '''
    
    #merge all the profile data
    profile_df = pd.merge(profile_general_info_cleaned_df,beds_rev_inpatient_days_df)
    #profile_df = pd.merge(profile_df,estimated_patient_volumes_df)
    return profile_df



In [123]:
#Parsing Financial Data
def get_financial_data(fn_name_path):
    #fn_name_path = "../web_data/ahd/financial/010131_financial_2018-08-14.xls"
    ccn = fn_name_path.split("/")[-1].split("_")[0]
    fn_xl = pd.ExcelFile(fn_name_path)
    nrows = fn_xl.book.sheet_by_index(0).nrows
    fn_df = pd.DataFrame()
    assets_found = True
    total_Assets_2017 = -1
    total_Assets_2016 = -1
    total_Liabilities_2017 = -1
    total_Liabilities_2016 = -1
    
    
    net_patient_revenue_2017=-1
    net_patient_revenue_2016 = -1
    net_income_or_loss_2017 = -1
    net_income_or_loss_2016 = -1
    # assets - liability
    #fn_fund_balance = fn_xl.parse(0, skipfooter= nrows-(30),skiprows=19).dropna(axis=1, how='all')
    try:
        fn_fund_balance = get_excel_section_as_df(fn_xl,"Assets","Income Statement")
        fn_fund_balance = fn_fund_balance.iloc[:,:3]
        fn_fund_balance.columns = ["Assets","2017","2016"]

        total_Assets_2017 = fn_fund_balance[fn_fund_balance.Assets == "Total Assets"]["2017"].values[0]
        total_Assets_2016 = fn_fund_balance[fn_fund_balance.Assets == "Total Assets"]["2016"].values[0]
        total_Liabilities_2017 = fn_fund_balance[fn_fund_balance.Assets == "Total Liabilities"]["2017"].values[0]
        total_Liabilities_2016 = fn_fund_balance[fn_fund_balance.Assets == "Total Liabilities"]["2016"].values[0]
    except:
        assets_found = False

    #fn_income_stmt = fn_xl.parse(0, skipfooter= nrows-(51),skiprows=36).dropna(axis=1, how='all')
    try:
        fn_income_stmt = get_excel_section_as_df(fn_xl,"Cost report status","Net Income or (Loss)",footer_offset=2)
        fn_income_stmt = fn_income_stmt.iloc[:,:3]
        fn_income_stmt.columns=["Income","2017","2016"]
        net_patient_revenue_2017 = fn_income_stmt[fn_income_stmt.Income == "Net Patient Revenues"]["2017"].values[0]
        net_patient_revenue_2016 = fn_income_stmt[fn_income_stmt.Income == "Net Patient Revenues"]["2016"].values[0]
        net_income_or_loss_2017 = fn_income_stmt[fn_income_stmt.Income == "Net Income or (Loss)"]["2017"].values[0]
        net_income_or_loss_2016 = fn_income_stmt[fn_income_stmt.Income == "Net Income or (Loss)"]["2016"].values[0]
    except:
        pass



    
    fn_df = fn_df.append([[total_Assets_2017,total_Assets_2016,total_Liabilities_2017,total_Liabilities_2016,
                          net_patient_revenue_2017,
                          net_patient_revenue_2016,
                          net_income_or_loss_2017,
                          net_income_or_loss_2016]])
    fn_df.columns = ["Total_Assets_2017","Total_Assets_2016","Total_Liabilities_2017","Total_Liabilities_2016",
                     "Net_Patient_Rev_2017",
                     "Net_Patient_Rev_2016",
                     "Net_Income_or_Loss_2017",
                     "Net_Income_or_Loss_2016"
                    ]
    fn_df["Provider_ID"] = ccn
    return fn_df

In [124]:
#Parsing department data
def get_dept_stats_data(dept_name_path):
    #dept_name_path = "../web_data/ahd/100001_departments_2018-08-15.xls"
    ccn = dept_name_path.split("/")[-1].split("_")[0]

    dept_xl = pd.ExcelFile(dept_name_path)
    nrows = dept_xl.book.sheet_by_index(0).nrows

    #ip_routine_service_cost = dept_xl.parse(0, skipfooter= nrows-(17),skiprows=13).dropna(axis=1, how='all')
    ip_routine_service_cost = get_excel_section_as_df(dept_xl,start_section_name="Inpatient Routine Service Cost Centers",end_section_name="Ancillary Service Cost Centers")
    ip_routine_service_cost = ip_routine_service_cost[ip_routine_service_cost["Unnamed: 0"] == "TOTAL"]
    ip_routine_service_cost = ip_routine_service_cost.drop(ip_routine_service_cost.columns[0],axis=1)
    ip_routine_service_cost.columns = ["total_ip_beds","ip_total_sq_feet","ip_total_costs","ip_gross_salaries","ip_charges","ratio_cost_per_chg","total_ip_days","cost_per_day"]
    ip_routine_service_cost["Provider_ID"] = ccn
    ip_routine_service_cost


    #ancillary_service_costs = dept_xl.parse(0, skipfooter= nrows-(48),skiprows=19).dropna(axis=1, how='all')
    ancillary_service_costs = get_excel_section_as_df(dept_xl,"Ancillary Service Cost Centers","Outpatient Service Cost Centers")

    ancillary_service_costs = ancillary_service_costs[ancillary_service_costs["Unnamed: 0"].isin(["Operating Room",
                                                                                                 "Anesthesiology",
                                                                                                 "Cardiac Catheterization",
                                                                                                 "Whole Blood / Packed RBC",
                                                                                                 "Blood Stor, Process, Trans",
                                                                                                 "Medical Supplies (charged)",
                                                                                                 "Drugs (charged)",
                                                                                                 "TOTAL"])]

    ancillary_service_costs = ancillary_service_costs[["Unnamed: 0","Gross Salaries","Total Costs","IP Charges","OP Charges","Ratio Cost/Chg"]]
    ancillary_service_costs["Total_Charges"] = ancillary_service_costs["IP Charges"] + ancillary_service_costs["OP Charges"]
    ancillary_service_costs = ancillary_service_costs.drop(["IP Charges", "OP Charges"],axis=1)
    ancillary_service_costs.columns = ["ancillary_service","ancillary_gross_salaries","ancillary_total_costs","ancillary_total_charges","ancillary_ratio_cost_per_chg"]
    ancillary_service_costs["ancillary_service"] = ancillary_service_costs["ancillary_service"].apply(replace_special_chars)

    ancillary_service_costs["Provider_ID"] = ccn
    ancillary_service_costs_df = ancillary_service_costs.pivot(index="Provider_ID",columns="ancillary_service").reset_index()
    ancillary_service_costs_df.columns = ['_'.join(col).strip() for col in ancillary_service_costs_df.columns.values]
    ancillary_service_costs_df.rename(columns={'Provider_ID_':'Provider_ID'},inplace=True)
    ancillary_service_costs_df



    #Outpatient Cost Centers
    #op_service_cost = dept_xl.parse(0, skipfooter= nrows-(57),skiprows=50).dropna(axis=1, how='all')
    op_service_cost = get_excel_section_as_df(dept_xl,start_section_name="Outpatient Service Cost Centers",end_section_name="Other Reimbursable Cost Centers")
    op_service_cost = op_service_cost[op_service_cost["Unnamed: 0"].isin(["Clinic","Emergency","TOTAL"])]
    op_service_cost["Total_Charges"] = op_service_cost["IP Charges"] + op_service_cost["OP Charges"]
    op_service_cost = op_service_cost.drop(["IP Charges","OP Charges","Square Feet"],axis=1)
    op_service_cost.columns = ["op_cost_center","op_gross_salaries","op_total_costs","op_ratio_cost_charge","total_charges"]
    op_service_cost["Provider_ID"] = ccn
    op_service_cost_df = op_service_cost.pivot(index="Provider_ID",columns="op_cost_center").reset_index()
    op_service_cost_df.columns = ['_'.join(col).strip() for col in op_service_cost_df.columns.values]
    op_service_cost_df.rename(columns={'Provider_ID_':'Provider_ID'},inplace=True)
    op_service_cost_df

    #Overall Cost To Charge Ratio
    #overall_cost_to_charge_ratio = dept_xl.parse(0, skipfooter= nrows-(97),skiprows=90).dropna(axis=1, how='all')
    overall_cost_to_charge_ratio = get_excel_section_as_df(dept_xl,"Overall Cost to Charge Ratio","General Service Costs")
    overall_cost_to_charge_ratio = overall_cost_to_charge_ratio[overall_cost_to_charge_ratio["Unnamed: 0"] == "TOTAL"]
    overall_cost_to_charge_ratio = overall_cost_to_charge_ratio.drop(["Unnamed: 0"],axis=1)
    overall_cost_to_charge_ratio_df = pd.DataFrame()
    overall_cost_to_charge_ratio_df = overall_cost_to_charge_ratio_df.append({'Provider_ID':ccn,
                                           'Total_Costs':overall_cost_to_charge_ratio["Total Costs"].values[0],
                                           'Total_Charges':overall_cost_to_charge_ratio["Total Charges"].values[0],
                                           'Total_ratio_cost_charge':overall_cost_to_charge_ratio["Ratio Cost/Chg"].values[0]},ignore_index=True)

    overall_cost_to_charge_ratio_df

    #Salaries and Other Costs of Pharmacy Staff
    #salaries_other_costs = dept_xl.parse(0, skipfooter= nrows-(114),skiprows=112).dropna(axis=1, how='all')
    salaries_other_costs = get_excel_section_as_df(dept_xl,"Salaries and Other Costs","Staffing")
    salaries_other_costs.columns = ["PH","DS","OC","Total_Costs","Paid_Hours","Avg_Hourly"]
    salaries_other_costs
    salaries_other_costs = salaries_other_costs[salaries_other_costs.PH == "Pharmacy"]
    pharm_salary_cost_df = pd.DataFrame()
    if len(salaries_other_costs) == 0:
        pharm_salary_cost_df = pharm_salary_cost_df.append({'Provider_ID':ccn,
                                                       'Pharmacy_Total_Cost':-1,
                                                       'Pharmacy_Paid_Hours':-1,
                                                       'Pharmacy_Avg_Hourly_Rate':-1},ignore_index=True)
    else:
        pharm_salary_cost_df = pharm_salary_cost_df.append({'Provider_ID':ccn,
                                                           'Pharmacy_Total_Cost':salaries_other_costs["Total_Costs"].values[0],
                                                           'Pharmacy_Paid_Hours':salaries_other_costs["Paid_Hours"].values[0],
                                                           'Pharmacy_Avg_Hourly_Rate':salaries_other_costs["Avg_Hourly"].values[0]},ignore_index=True)
    pharm_salary_cost_df


    #Staffing (Full time employees)
    #staffing_numbers = dept_xl.parse(0, skipfooter= nrows-(133),skiprows=120).dropna(axis=1, how='all')
    staffing_numbers = get_excel_section_as_df(dept_xl,"Staffing",end_section_name='',offset=1)
    staffing_numbers_df = pd.DataFrame()

    staffing_numbers_df = staffing_numbers_df.append({'Provider_ID' : ccn,
                                                     'Number_Of_Pharmacists':staffing_numbers[staffing_numbers.Positions == 'Pharmacists:'].FTEs.values[0] ,
                                                     'Total_Number_Of_FTE' :staffing_numbers.FTEs.sum() },ignore_index=True)
    #merge all department data frames
    dept_df = pd.merge(ip_routine_service_cost,ancillary_service_costs_df,on="Provider_ID")
    dept_df = pd.merge(dept_df,op_service_cost_df,on="Provider_ID")
    dept_df = pd.merge(dept_df,overall_cost_to_charge_ratio_df,on="Provider_ID")
    dept_df = pd.merge(dept_df,pharm_salary_cost_df,on="Provider_ID")
    dept_df = pd.merge(dept_df,staffing_numbers_df,on="Provider_ID")
    return dept_df

In [125]:
def get_inpatient_data (inpatient_file_path):
    #inpatient_file_path = "../web_data/ahd/510001_inpatient_2018-08-14.xls"
    ccn = inpatient_file_path.split("/")[-1].split("_")[0]
    inpatient_xl = pd.ExcelFile(inpatient_file_path)
    nrows = inpatient_xl.book.sheet_by_index(0).nrows

    
    #Trend Report - Case Mix Index
    ip_trend_report = get_excel_section_as_df(inpatient_xl,"Trend Report","Statistics for the Top 20 Base MS-DRGs",offset=1)
    cmi_df = ip_trend_report[ip_trend_report["Inpatient Utilization Statistics"] == "Case Mix Index"]
    try:
        cmi_recent_value = get_value_if_exists(cmi_df,1,2)[0]
    except:
        cmi_recent_value = -1


    #Stats by Medical Service
    medical_service_stats = get_excel_section_as_df(inpatient_xl,"Statistics by Medical Service",offset=1)
    cardiology_stats = get_value_if_exists(medical_service_stats[medical_service_stats["Unnamed: 0"] == "Cardiology"],1,6,reshape=True)
    cardiovascular_surgery_stats = get_value_if_exists(medical_service_stats[medical_service_stats["Unnamed: 0"] == "Cardiovascular Surgery"],1,6,reshape=True)
    medicine_stats = get_value_if_exists(medical_service_stats[medical_service_stats["Unnamed: 0"] == "Medicine"],1,6,reshape=True)
    surgery_stats = get_value_if_exists(medical_service_stats[medical_service_stats["Unnamed: 0"] == "Surgery"],1,6,reshape=True)
    vascular_surgery_stats = get_value_if_exists(medical_service_stats[medical_service_stats["Unnamed: 0"] == "Vascular Surgery"],1,6,reshape=True)
    medical_service_total_stats = get_value_if_exists(medical_service_stats[medical_service_stats["Unnamed: 0"] == "TOTAL"],1,6,reshape=True)

    medical_service_stats_df = pd.DataFrame(cardiology_stats).append(pd.DataFrame(cardiovascular_surgery_stats)).append(pd.DataFrame(medicine_stats)).append(pd.DataFrame(surgery_stats)).append(pd.DataFrame(vascular_surgery_stats)).append(pd.DataFrame(medical_service_total_stats))
    medical_service_stats_df = medical_service_stats_df.reset_index().drop("index",axis=1)
    medical_service_stats_df.columns = ["no_of_inpatients","ip_avg_stay_in_days","ip_avg_charges","ip_avg_cost","ip_cmi","ip_adjusted_avg_cost"]
    medical_service_stats_df = medical_service_stats_df.drop(["ip_avg_cost","ip_cmi"],axis=1)
    medical_service_stats_df["med_service"] = ["Cardiology","Cardiovascular_Surgery","Medicine","Surgery","Vascular_Surgery","Inpatient_Medical_Service_Total"]
    medical_service_stats_df["Provider_ID"] = ccn

    medical_service_stats_df = medical_service_stats_df.pivot(index="Provider_ID",columns="med_service").reset_index()
    medical_service_stats_df.columns = ['_'.join(col).strip() for col in medical_service_stats_df.columns.values]
    medical_service_stats_df.rename(columns={'Provider_ID_':'Provider_ID'},inplace=True)
    medical_service_stats_df["cmi_recent_value"] = cmi_recent_value
    
    return medical_service_stats_df

In [126]:
def get_outpatient_data(outpatient_file_path):
    #Parsing Outpatient stats 
    #outpatient_file_path = "../web_data/ahd/510001_outpatient_2018-08-14.xls"
    ccn = outpatient_file_path.split("/")[-1].split("_")[0]
    outpatient_xl = pd.ExcelFile(outpatient_file_path)
    nrows = outpatient_xl.book.sheet_by_index(0).nrows
    
    if nrows <80: #Outpatient file does not have the needed data
        op_cols = ['Provider_ID', 'Number_Of_OP_Claims_Anesthesia',
       'Number_Of_OP_Claims_Blood_Storage_and_Processing',
       'Number_Of_OP_Claims_Cardiac_Cath_Lab',
       'Number_Of_OP_Claims_Cardiology',
       'Number_Of_OP_Claims_Drugs_Requiring_Specific_Identification',
       'Number_Of_OP_Claims_Emergency_Room', 'Number_Of_OP_Claims_IV_Therapy',
       'Number_Of_OP_Claims_Medical_Surgical_Supplies',
       'Number_Of_OP_Claims_Pharmacy', 'OP_Avg_Charge_Anesthesia',
       'OP_Avg_Charge_Blood_Storage_and_Processing',
       'OP_Avg_Charge_Cardiac_Cath_Lab', 'OP_Avg_Charge_Cardiology',
       'OP_Avg_Charge_Drugs_Requiring_Specific_Identification',
       'OP_Avg_Charge_Emergency_Room', 'OP_Avg_Charge_IV_Therapy',
       'OP_Avg_Charge_Medical_Surgical_Supplies', 'OP_Avg_Charge_Pharmacy',
       'OP_Avg_Cost_Anesthesia', 'OP_Avg_Cost_Blood_Storage_and_Processing',
       'OP_Avg_Cost_Cardiac_Cath_Lab', 'OP_Avg_Cost_Cardiology',
       'OP_Avg_Cost_Drugs_Requiring_Specific_Identification',
       'OP_Avg_Cost_Emergency_Room', 'OP_Avg_Cost_IV_Therapy',
       'OP_Avg_Cost_Medical_Surgical_Supplies', 'OP_Avg_Cost_Pharmacy']

        empty_op_df = pd.DataFrame(data= np.full((1,len(op_cols)),-1) ,columns=op_cols)
        empty_op_df["Provider_ID"] = ccn
        return empty_op_df
        

    #Service Stats
    op_claims_stats = get_excel_section_as_df(outpatient_xl,"Service Statistics",offset=1)
    op_claims_stats = op_claims_stats[op_claims_stats.Service.isin(["Pharmacy","IV Therapy","Medical Surgical Supplies",
                                                                   "Blood Storage and Processing",
                                                                   "Anesthesia",
                                                                   "Emergency Room",
                                                                   "Cardiology",
                                                                   "Cardiac Cath Lab",
                                                                  "Drugs Requiring Specific Identification"])]

    op_claims_stats = op_claims_stats.drop(["Units of Service","Average Payment","Service Mix Index - SMI"],axis=1)
    op_claims_stats["Service"] = op_claims_stats["Service"].apply(lambda x : x.replace(' ','_'))


    op_pharmacy_stats = get_value_if_exists(op_claims_stats[op_claims_stats["Service"] == "Pharmacy"],1,3,reshape=True)
    op_iv_therapy_stats = get_value_if_exists(op_claims_stats[op_claims_stats["Service"] == "IV_Therapy"],1,3,reshape=True)
    op_medical_surg_supplies = get_value_if_exists(op_claims_stats[op_claims_stats["Service"] == "Medical_Surgical_Supplies"],1,3,reshape=True)
    op_anesth = get_value_if_exists(op_claims_stats[op_claims_stats["Service"] == "Anesthesia"],1,3,reshape=True)
    op_blood_storage_processing = get_value_if_exists(op_claims_stats[op_claims_stats["Service"] == "Blood_Storage_and_Processing"],1,3,reshape=True)
    op_emergency_room = get_value_if_exists(op_claims_stats[op_claims_stats["Service"] == "Emergency_Room"],1,3,reshape=True)
    op_cardio = get_value_if_exists(op_claims_stats[op_claims_stats["Service"] == "Cardiology"],1,3,reshape=True)
    op_cardiac_lab = get_value_if_exists(op_claims_stats[op_claims_stats["Service"] == "Cardiac_Cath_Lab"],1,3,reshape=True)
    op_drugs_specific = get_value_if_exists(op_claims_stats[op_claims_stats["Service"] == "Drugs_Requiring_Specific_Identification"],1,3,reshape=True)

    op_claims_stats_df = pd.DataFrame(op_pharmacy_stats)
    op_claims_stats_df = op_claims_stats_df.append(pd.DataFrame(op_iv_therapy_stats))
    op_claims_stats_df = op_claims_stats_df.append(pd.DataFrame(op_medical_surg_supplies))
    op_claims_stats_df = op_claims_stats_df.append(pd.DataFrame(op_anesth))
    op_claims_stats_df = op_claims_stats_df.append(pd.DataFrame(op_blood_storage_processing))
    op_claims_stats_df = op_claims_stats_df.append(pd.DataFrame(op_emergency_room))
    op_claims_stats_df = op_claims_stats_df.append(pd.DataFrame(op_cardio))
    op_claims_stats_df = op_claims_stats_df.append(pd.DataFrame(op_cardiac_lab))
    op_claims_stats_df = op_claims_stats_df.append(pd.DataFrame(op_drugs_specific))
    op_claims_stats_df.columns = ["Number_Of_OP_Claims","OP_Avg_Charge","OP_Avg_Cost"]
    op_claims_stats_df["OP_Med_Service"] = ["Pharmacy","IV_Therapy",
                                            "Medical_Surgical_Supplies",
                                            "Anesthesia",
                                            "Blood_Storage_and_Processing",
                                            "Emergency_Room",
                                            "Cardiology",
                                            "Cardiac_Cath_Lab",
                                            "Drugs_Requiring_Specific_Identification"]

    op_claims_stats_df["Provider_ID"] = ccn
    op_claims_stats_df = op_claims_stats_df.pivot(index="Provider_ID",columns="OP_Med_Service").reset_index()
    op_claims_stats_df.columns = ['_'.join(col).strip() for col in op_claims_stats_df.columns.values]
    op_claims_stats_df.rename(columns={'Provider_ID_':'Provider_ID'},inplace=True)
    return op_claims_stats_df


In [ ]:
print(op_drugs_specific)
op_claims_stats

In [137]:
prefix = "../web_data/ahd/definitive"
c = 0
file_path_formatter = prefix + "/{1}/{0}_{1}.xls"
hospital_merged_df = pd.DataFrame()
number_of_invalid_profiles = 0
for profile_file_path in glob.glob(prefix + "/profile/*.xls"):
    if c % 100 == 0:
        print(c)
    c = c +1
    if check_valid_profile(profile_file_path) == False:
        number_of_invalid_profiles = number_of_invalid_profiles + 1
        print('Invalid Profile '  + profile_file_path + ' ' + str(number_of_invalid_profiles) )

        continue
    #print(c)
    #print (profile_file_path)
    provider_id = profile_file_path.split("/")[-1].split('_')[0]
    financial_file_path = file_path_formatter.format(provider_id,"financial")
    departments_file_path = file_path_formatter.format(provider_id,"departments")
    inpatient_file_path = file_path_formatter.format(provider_id,"inpatient")
    outpatient_file_path = file_path_formatter.format(provider_id,"outpatient")
    profile_df = get_profile_df(profile_file_path)
    try:
        financial_df = get_financial_data(financial_file_path)
        dept_df = get_dept_stats_data(departments_file_path)
        inpatient_df = get_inpatient_data(inpatient_file_path)  

        outpatient_df = get_outpatient_data(outpatient_file_path)
        tmp_df = pd.merge(profile_df,financial_df,on="Provider_ID")
        tmp_df = pd.merge(tmp_df,dept_df,on="Provider_ID")
        tmp_df = pd.merge(tmp_df,inpatient_df,on="Provider_ID")
        tmp_df = pd.merge(tmp_df,outpatient_df,on="Provider_ID")
        #print(len(tmp_df))
        #print(profile_df["Provider_ID"])
        #print(financial_df["Provider_ID"])
        if len(hospital_merged_df) == 0:
            hospital_merged_df=tmp_df
        else:
            hospital_merged_df = pd.concat([hospital_merged_df,tmp_df])
    except FileNotFoundError:
        print('File Not Found')
        with open("missing_downloads.csv","a") as myfile:
            myfile.write("{}\n".format(provider_id))
        pass
    except Exception as e:
        print('Problem with ' + provider_id)
        print(e)
    #break

0
Invalid Profile ../web_data/ahd/definitive/profile/222025_profile.xls 1
Invalid Profile ../web_data/ahd/definitive/profile/52002F_profile.xls 2
Invalid Profile ../web_data/ahd/definitive/profile/10015F_profile.xls 3
Invalid Profile ../web_data/ahd/definitive/profile/39012F_profile.xls 4
Invalid Profile ../web_data/ahd/definitive/profile/364045_profile.xls 5
Invalid Profile ../web_data/ahd/definitive/profile/670104_profile.xls 6
Invalid Profile ../web_data/ahd/definitive/profile/05031F_profile.xls 7
Problem with 524041
cannot reshape array of size 5 into shape (1,6)
Invalid Profile ../web_data/ahd/definitive/profile/05130F_profile.xls 8
Invalid Profile ../web_data/ahd/definitive/profile/024001_profile.xls 9
Invalid Profile ../web_data/ahd/definitive/profile/05114F_profile.xls 10
Invalid Profile ../web_data/ahd/definitive/profile/424013_profile.xls 11
100
Invalid Profile ../web_data/ahd/definitive/profile/054114_profile.xls 12
Problem with 061991
cannot reshape array of size 5 into sha

In [141]:
len(hospital_merged_df)


,Type_Of_Facility,Is_Low_Volume_Hospital,Total_Emp,Total_Discharges,Urban_Rural,Provider_ID,beds,revenue,Total_Inpatient_Days,Gross_Patient_Revenue,...,OP_Avg_Charge_Pharmacy,OP_Avg_Cost_Anesthesia,OP_Avg_Cost_Blood_Storage_and_Processing,OP_Avg_Cost_Cardiac_Cath_Lab,OP_Avg_Cost_Cardiology,OP_Avg_Cost_Drugs_Requiring_Specific_Identification,OP_Avg_Cost_Emergency_Room,OP_Avg_Cost_IV_Therapy,OP_Avg_Cost_Medical_Surgical_Supplies,OP_Avg_Cost_Pharmacy
0,Psychiatric,N,97,"1,921",Urban,104049,72.0,15758247.0,11537,19905290,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,Rehabilitation,N,117,979,Urban,673061,45.0,25587692.0,12421,51282094,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,Rehabilitation,N,171,"1,382",Urban,313035,60.0,20376757.0,18573,40590535,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,Psychiatric,N,1564,256,Urban,334015,440.0,130543.0,130543,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,Psychiatric,N,187,"3,280",Urban,064009,76.0,37902000.0,25263,46702334,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [ ]:
d0 = get_profile_df("../web_data/ahd/profile/100168_profile.xls")
d1 = get_financial_data("../web_data/ahd/financial/100168_financial.xls")
len(hospital_merged_df["Provider_ID"])
pd.DataFrame(hospital_merged_df.columns).to_clipboard()
#hospital_merged_df["Provider_ID"]
inpatient_df.columns
hospital_merged_df = hospital_merged_df.fillna(-1)
hospital_merged_df = hospital_merged_df.drop_duplicates()
coverage_df = pd.DataFrame(columns=["col_name","coverage_percentage"])
for c in list(hospital_merged_df.columns):
    na_values = len(hospital_merged_df[hospital_merged_df[c] == -1])
    covered_values_perc = ((len(hospital_merged_df) - na_values)/len(hospital_merged_df)) * 100
    coverage_df = coverage_df.append({'col_name':c,'coverage_percentage':covered_values_perc},ignore_index=True)


In [121]:
#d0 = get_profile_df("../web_data/ahd/profile/100168_profile.xls")

d0 = get_profile_df("../web_data/ahd/definitive/profile/452095_profile.xls")
check_valid_profile("../web_data/ahd/definitive/profile/452095_profile.xls")
#../web_data/ahd/definitive/profile/162001_profile.xls


True

In [ ]:
coverage_df.to_clipboard(index=False)

In [ ]:
#hospital_merged_df.groupby("Provider_ID").count()["Type_Of_Facility"].reset_index().sort_values(by="Type_Of_Facility",ascending=False)

In [142]:
hospital_merged_df.to_csv("Hospitals_Features/definitive_ahd_features.csv",index=False)

In [136]:
#definitive data
def_df = pd.read_csv("DefinitiveAhdDifference.csv")
def_df[[""]]
#hospital_merged_df

,Case_Mix_Index,#_of_Hospitals,Routine_Service_Days,Routine_Service_Beds,Total_Acute_Beds,Est_#_of_Outpatient_Visits,Est_#_of_Inpatient_Surgeries,Total_Acute_Days,Average_Length_of_Stay,Number_of_Operating_Rooms,...,City,State,Definitive_ID,Provider_Number,IDN,IDN_Parent,Net_Patient_Revenue,NoofDischarges,NoofStaffedBeds,ZsIdentified
0,NaN,NaN,NaN,NaN,NaN,NaN,3585.0,NaN,NaN,NaN,...,Travis Afb,CA,551551,05015F,US Air Force Medicine,US Department of Defense Military Health System,NaN,NaN,NaN,Untiered
1,NaN,NaN,NaN,NaN,NaN,NaN,481.0,NaN,NaN,NaN,...,Hampton,VA,575034,49005F,US Air Force Medicine,US Department of Defense Military Health System,NaN,NaN,NaN,Untiered
2,2.20,NaN,50034.0,224.0,262.0,49682.0,2900.0,65550.0,4.1,11.0,...,Glendale,AZ,176,030094,Abrazo Community Health Network,Tenet Healthcare,3.098807e+08,15821.0,262.0,Tier 4
3,1.82,NaN,21501.0,97.0,125.0,25795.0,1886.0,29967.0,4.1,7.0,...,Phoenix,AZ,161,030030,Abrazo Community Health Network,Tenet Healthcare,1.078541e+08,7265.0,125.0,Tier 4
4,1.55,NaN,18027.0,113.0,127.0,24431.0,1126.0,22055.0,3.8,8.0,...,Phoenix,AZ,170,030083,Abrazo Community Health Network,Tenet Healthcare,7.635006e+07,5863.0,127.0,Untiered
5,1.60,NaN,36654.0,159.0,179.0,35811.0,1607.0,45087.0,3.9,8.0,...,Goodyear,AZ,182,030110,Abrazo Community Health Network,Tenet Healthcare,1.813822e+08,11502.0,179.0,Tier 4
6,1.02,NaN,20711.0,68.0,68.0,62251.0,NaN,20711.0,14.8,NaN,...,Bangor,ME,840554,204006,Eastern Maine Healthcare System (EMHS),Eastern Maine Healthcare System (EMHS),5.323983e+07,1395.0,68.0,Tier 4
7,1.14,NaN,3071.0,27.0,27.0,NaN,NaN,3071.0,13.2,NaN,...,Crowley,LA,6394,193067,Lafayette General Health,NaN,5.872438e+06,232.0,27.0,Untiered
8,1.02,NaN,18603.0,54.0,54.0,NaN,NaN,18603.0,8.9,NaN,...,Lafayette,LA,5210,194044,Acadia Healthcare,NaN,1.124803e+07,2088.0,54.0,Untiered
9,1.05,NaN,7863.0,24.0,24.0,667.0,195.0,7863.0,8.7,NaN,...,Lafayette,LA,5216,194078,Acadia Healthcare,NaN,5.307827e+06,908.0,24.0,Untiered


In [102]:
#convert from DMS to DD Lat/Long
import re

def dms2dd(degrees, minutes, seconds, direction):
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction == 'W' or direction == 'S':
        dd *= -1
    return dd;

def dd2dms(deg):
    d = int(deg)
    md = abs(deg - d) * 60
    m = int(md)
    sd = (md - m) * 60
    return [d, m, sd]

def parse_dms(dms):
    parts = re.split('[^\d\w]+', dms)
    try:
        lat = dms2dd(parts[0], parts[1], 0, parts[2])
    except:
        print(dms)
        return ''
    return (lat)

In [145]:
#ADDING MORE FEATURES ... 
#Scrapping GPOs for the providers' profiles 
provider_gpo_df = pd.DataFrame()
lat_long_df = pd.DataFrame()

#profile_general_info_df = profile_xl.parse(0, skipfooter= nrows-(43),skiprows=19).dropna(axis=1, how='all')
#profile_general_info_df = get_excel_section_as_df(profile_xl,"CMS Certification Number:").iloc[:,0:2]
counter = 1
prefix = "../web_data/ahd/definitive"
for profile_file_path in glob.glob(prefix + "/profile/*.xls"):
    profile_xl = pd.ExcelFile(profile_file_path)
    current_gpo = ''
    lat=''
    longit=''
    if counter %500 == 0:
        print (counter)
    provider_id = profile_file_path.split("/")[-1].split('_')[0]
    if check_valid_profile(profile_file_path) == False:
        continue
    
    try:
        current_gpo = get_excel_section_as_df(profile_xl,"Primary",offset=-1,end_section_name="Accountable Care Organizations (ACOs)")["Primary"].iloc[0].split('\n')[0]
        long_lat = get_excel_section_as_df(profile_xl,"Latitude / Longitude:",offset=-2,footer_offset=-1,end_section_name="Medicare Administrative Contractor (MAC):").iloc[0][1]
        lat = parse_dms(long_lat.split("/")[0].strip())
        longit = parse_dms(long_lat.split("/")[1].strip())
    except:
        current_gpo = 'NA'
        #print(long_lat)

    provider_gpo_df = provider_gpo_df.append({'Provider_ID':provider_id , 'GPO':current_gpo},ignore_index=True)
    lat_long_df = lat_long_df.append({'Provider_ID':provider_id , 'lat':lat,'longit':longit},ignore_index=True)
    counter = counter + 1

500
1000


In [146]:
provider_gpo_df["GPO_Text__c_Vizient"] = provider_gpo_df.GPO.apply(lambda x : 1 if x == "Vizient Inc." else 0)
provider_gpo_df["Provider_ID"] = provider_gpo_df["Provider_ID"].astype(str)
provider_gpo_df["Provider_ID"] = provider_gpo_df["Provider_ID"].apply(lambda x : x.zfill(6))
lat_long_df.to_csv("Hospitals_Features/ahd_provider_lat_long.csv",index=False)
provider_gpo_df.to_csv("Hospitals_Features/definitve_ahd_provider_gpo.csv",index=False)

In [147]:
provider_gpo_df.groupby('GPO').count()

,Provider_ID,GPO_Text__c_Vizient
GPO,,
CHC Supply Trust,1,1
Capstone Health Alliance,1,1
Cardinal Health Puerto Rico,3,3
Cardinal Health Specialty Pharmaceutical Services,8,8
Carolina Shared Services,1,1
CoreTrust Purchasing Group,8,8
DSSI,1,1
Does Not Use A GPO,21,21
Douglas County Purchasing,1,1


In [ ]:
#get_outpatient_data(outpatient_file_path="../web_data/ahd/outpatient/010001_outpatient.xls").T

In [52]:
profile_name_path = "../web_data/ahd/profile/100001_profile.xls"
profile_xl = pd.ExcelFile(profile_name_path)
lat_long = get_excel_section_as_df(profile_xl,"Latitude / Longitude:",offset=-2,footer_offset=-1,end_section_name="Medicare Administrative Contractor (MAC):").iloc[0][1]
lat_long
#long_lat_df.columns = [""]

"30 20'N / 81 39'W"

In [101]:
parse_dms("""44 57'N / 93 15'W""".split("/")[1].strip())
#print(dms2dd(30,20,0,'N'))
#print(dms2dd(81,39,0,'W'))


-93.25

In [114]:
tp_shape_values_df = pd.read_csv("Hospitals_Features/tp_shape_values_df.csv")
#pd.merge(tp_shape_values_df,long_lat_df)
tp_shape_values_df["Provider_ID"] = tp_shape_values_df["Provider_ID"].astype(str)
lat_long_df["Provider_ID"] = lat_long_df["Provider_ID"].astype(str)
pd.merge(tp_shape_values_df,lat_long_df).to_csv("Hospitals_Features/tp_shape_values_df_with_lat_long.csv",index=False)